In [1]:
from pydataset import data
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection

import sklearn.preprocessing

In [2]:
def split(df, stratify_by=""):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    return train, validate, test

In [3]:
# Exercise 1
# a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.
# b. Create a column named price_per_person. This should be the total bill divided by the party size.
# c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

df = data('tips')

# Rename the size column because .size is a built-in Pandas attribute
df = df.rename(columns={'size': 'number_of_people'})

# Bracket notation to create new columns
df["tip_percentage"] = df.tip / df.total_bill
df["price_per_person"] = df.total_bill / df.number_of_people

In [4]:
# For this specific exercise, we're only focusing on the numeric features
df = df[["total_bill", "tip", "number_of_people", "tip_percentage", "price_per_person"]]

In [5]:
# Split the data

train, validate, test = split(df, stratify_by="tip")

In [6]:
target = "tip"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

In [7]:
# Scale
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler
scaler.fit(X_train)

# Use the scaler to transform train, validate, test
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Exercise 1c
# Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?
# I reason that the tip_percentage will be the best predictor of tip, since it's a multiplier...

In [9]:
# Exercise 1d
# Use numeric features to predict tip_amount
# Use select-K-best and RFE to select the top 2 features

k = 2

# Let's start with Select K Best
# Make the thing
kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=2)

# fit the thing
kbest.fit(X_train, y_train)

# use the thing
kbest_features = X_train.columns[kbest.get_support()].tolist()

print("KBest's 2 best features are", kbest_features)

KBest's 2 best features are ['total_bill', 'number_of_people']


In [10]:
# Now let's do the RFE

# Make the thing(s)
lm = sklearn.linear_model.LinearRegression()
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)

# Fit the thing
rfe.fit(X_train, y_train)

# use the thing
rfe_columns = X_train.columns[rfe.support_].tolist()
rfe_columns

['number_of_people', 'tip_percentage']

In [11]:
# Exercise 2
# Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

def select_kbest(X, y, k):
    kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=k)
    kbest.fit(X, y)
    return X.columns[kbest.get_support()]

select_kbest(X_train, y_train, 2)

Index(['total_bill', 'number_of_people'], dtype='object')

In [12]:
# Exercise 3
# Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

def select_rfe(X, y, k):
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)
    rfe.fit(X, y)
    return X.columns[rfe.support_]

select_rfe(X_train, y_train, 2)

Index(['number_of_people', 'tip_percentage'], dtype='object')

In [13]:
# Exercise 4
# Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [14]:
# Split the data
train, validate, test = split(swiss)

# Setup X and y
X_train = train.drop(columns='Fertility')
y_train = train.Fertility

X_validate = validate.drop(columns='Fertility')
y_validate = validate.Fertility

X_test = test.drop(columns='Fertility')
y_test = test.Fertility

In [15]:
# Scale the data
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler
scaler.fit(X_train)

# Use the scaler to transform train, validate, test
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Find the top 3 features using kbest
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

In [17]:
# Find the top 3 features using RFE
select_rfe(X_train, y_train, 3)

Index(['Agriculture', 'Examination', 'Infant.Mortality'], dtype='object')